<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Dependencies" data-toc-modified-id="Dependencies-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Dependencies</a></span></li><li><span><a href="#Initiating-the-dataframes" data-toc-modified-id="Initiating-the-dataframes-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Initiating the dataframes</a></span></li><li><span><a href="#Scraping-HLTV-into-the-dataframes" data-toc-modified-id="Scraping-HLTV-into-the-dataframes-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Scraping HLTV into the dataframes</a></span></li></ul></div>

# Extensive HLTV Scrape

## Dependencies
I will use pandas to save the data and export it as a csv file. I will use BeautifulSoup to parse the html content of the hltv pages. I will use requests to get the hltv pages. time for pausing the program between requests and re for regular expression matching.

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import re


## Initiating the dataframes
 - resultsdf for the results of every played map.
 - picksdf for the mappicks for every team in every veto process.
 - bansdf for the mapbans for every team in every veto process.

In [3]:
resultsdf = pd.DataFrame(columns=['matchcode', 'map', 'team1', 'team2', 'score1', 'score2', 'event', 'stars', 'date'])
picksdf = pd.DataFrame(columns=['matchcode', 'team', 'map'])
bansdf = pd.DataFrame(columns=['matchcode', 'team', 'map'])

## Scraping HLTV into the dataframes

In [9]:
def scrapePage(soup, resultsdf, picksdf, bansdf):
    patternBan = re.compile("removed")
    patternPick = re.compile("picked")
    sublists = soup.find_all("div", {"class": "results-sublist"})
    
    #for the first page when hltv displays feaured results
    #for sublist in sublists[1:]:
    for sublist in sublists:
        try:
            date = getDate(sublist.find("span", {"class":"standard-headline"}).text.strip())
            results = sublist.find_all("div", {"class":"result-con"})
            for result in results:
                #time.sleep(.5)
                url = "https://www.hltv.org" + result.find("a", {"class":"a-reset"})['href']
                team1Name = result.find("div", {"class":"team1"}).text.strip()
                team2Name = result.find("div", {"class":"team2"}).text.strip()     
                event = result.find("span", {"class":"event-name"}).text.strip()
                matchCode = result.find("a", {"class":"a-reset"})['href'].split('/')[2]
                stars = str(len(result.find_all("i", {"class":"fa fa-star star"})))
                innersoup = BeautifulSoup(requests.get(url).content, "lxml")
                maps = innersoup.find_all("div", {"class": "mapholder"})
                for result in maps:
                    mapname = result.find("div", {'class':'mapname'}).text.strip()
                    results = result.find("div", {'class': 'results'})
                    if results is not None:
                        score1 = results.find_all("span")[0].text.strip()
                        score2 = results.find_all("span")[2].text.strip()
                        resultsdf = resultsdf.append({'matchcode': matchCode, 'map': mapname, 'team1': team1Name, 'team2':team2Name, 'score1':score1, 'score2':score2, 'event':event, 'stars':stars, 'date':date}, ignore_index=True)
                try:
                    vetos = innersoup.find_all("div",{'class':'standard-box veto-box'})[1].find('div').find_all('div')
                    for veto in vetos:
                        string = veto.text.strip()
                        if patternBan.search(string):
                            splitstring = string.split(' removed ')
                            bansdf = bansdf.append({'matchcode': matchCode, 'team': splitstring[0][3:], 'map':splitstring[-1]}, ignore_index=True)
                        elif patternPick.search(string):
                            splitstring = string.split(' picked ')
                            picksdf = picksdf.append({'matchcode': matchCode, 'team': splitstring[0][3:], 'map':splitstring[-1]}, ignore_index=True)
                except IndexError as e:
                    print("No vetos for: ", matchCode)
        except IOError as e:
            print (e)
    return (resultsdf, picksdf, bansdf)
            
            

months = ["default", "January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

#string format = 'Results for September 10th 2018' returned format = '2018-09-10'
def getDate(string):
    splitString = string.split(' ')
    year = str(splitString[4])
    month = months.index(str(splitString[2]))
    if (month < 10):
        month = "0"+str(month)
    day = int(splitString[3][:-2])
    if (day < 10):
        day = "0"+str(day)
    return str(year) + "-" + str(month) + "-" + str(day)
                                                   
def doNothing():
    return


url = 'https://www.hltv.org/results?offset='
#for i in range(0,1):
    #time.sleep(1)  
    #(resultsdf, picksdf, bansdf) = scrapePage(BeautifulSoup(requests.get(url+str(i)+'00').content, "lxml"), resultsdf, picksdf, bansdf)
    #print('Scraped till page: ', i)
